In [1]:
import os
import cv2
import numpy as np
import json
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
DATA_PATH = os.path.join(os.getcwd(), '../data/3_pts')
print(DATA_PATH)
PROCESSED_DATA_PATH = os.path.join(os.getcwd(), '../processed/3_pts')
if not os.path.exists(PROCESSED_DATA_PATH):
    os.makedirs(PROCESSED_DATA_PATH)
if not os.path.exists(os.path.join(PROCESSED_DATA_PATH,'images')):
    os.makedirs(os.path.join(PROCESSED_DATA_PATH,'images'))

/home/boomalope/nerf_estimator/src/nerf_estimator/nerf_vision/scripts/../data/3_pts


In [3]:
#Gazebo Param: 
#Definition: https://github.com/ros-simulation/gazebo_ros_pkgs/blob/398cde933cba5d72921055d57d4faa47c4445654/gazebo_plugins/src/gazebo_ros_camera_utils.cpp#L524

In [4]:
#Camera parameters
#Definitions: https://docs.nerf.studio/quickstart/data_conventions.html

fl_x = 762.72 # focal length in x
fl_y = 762.72# focal length in y
w = 1280 # image width
h = 720 # image height
cx = w//2 # optical center in x
cy = h//2 # optical center in y
camera_model = "OPENCV" # camera model
k1 = 0 # radial distortion coefficient k1
k2 = 0 # radial distortion coefficient k2
k3 = 0 # radial distortion coefficient k3
p1 = 0 # tangential distortion coefficient p1
p2 = 0 # tangential distortion coefficient p2

In [5]:
def from_tf_to_matrix(tf):
    #tf is the frame of the robot relative to the camera
    xyz = tf[:3]
    q = tf[3:7]
    goal = tf[7:]

    #Rotation matrix
    r = Rotation.from_quat(q)
    r = r.as_matrix()

    #Additional rotation  matrix to shift camera from +x to -z direction and make the x axis point to the right
    A = Rotation.from_euler('YZX', [-90, -90, 0], degrees=True).as_matrix()

    #Translation matrix in the world frame
    t = np.zeros((4,4))
    t[:3,:3] = np.matmul(r.T,A)
    t[:3,3] = -np.matmul(r.T,xyz)
    t[3,3] = 1

    print(t[:3,3])
    print(Rotation.from_matrix(t[:3,:3]).as_euler('xyz', degrees=True))
    
    return t

In [8]:
data_cnt = 0
frames = []
while True:
    #Get data paths
    data_img = os.path.join(DATA_PATH, '{}.png'.format(data_cnt))
    data_tf= os.path.join(DATA_PATH, '{}.txt'.format(data_cnt))

    if not os.path.exists(data_img):
        break
    
    #Get tf as np array
    tf = np.loadtxt(data_tf, delimiter=',')


    file_path = "images/{}.png".format(data_cnt)
    transform_matrix = from_tf_to_matrix(tf)

    #Append to frames
    frames.append({
        "file_path": file_path,
        "transform_matrix": transform_matrix.tolist(),
    })

    #Save image in processed folder
    img = cv2.imread(data_img)
    #Change img from BGR to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    cv2.imwrite(os.path.join(PROCESSED_DATA_PATH, file_path), img)
    
    data_cnt += 1       

[-1.76282162  0.05828205  2.00014581]
[ 6.00278319e+01  2.74204047e-02 -9.17332759e+01]
[-1.76281524  0.04851942  2.00015662]
[ 6.00275165e+01  4.12582897e-02 -9.14316824e+01]
[-1.76260982  0.04110801  2.00022155]
[ 6.00253497e+01  2.08633336e-02 -9.11677238e+01]
[-1.75008684  0.21270152  2.00010249]
[ 6.00292664e+01  3.80261982e-02 -9.67796697e+01]
[-1.71732551  0.39652672  2.00023917]
[ 6.00248233e+01  3.80472318e-02 -1.02851431e+02]
[-1.67745086  0.54240693  2.00034277]
[ 6.00214593e+01  3.94862454e-02 -1.07775703e+02]
[-1.62863155  0.67500395  2.00069019]
[ 6.00101735e+01  4.13164017e-02 -1.12376142e+02]
[-1.56572299  0.81122675  2.00056149]
[ 6.00143604e+01  4.29324119e-02 -1.17258854e+02]
[-1.46822713  0.97576663  2.00084364]
[ 6.00051877e+01  4.13994566e-02 -1.23471532e+02]
[-1.40663921  1.061769    2.00110252]
[ 5.99967837e+01  4.66292056e-02 -1.26912131e+02]
[-1.3011248   1.18961554  2.00090904]
[ 6.00030706e+01  4.52553103e-02 -1.32311429e+02]
[-1.19099285  1.30007133  2.0011

In [7]:
transform_dictionary = {}
transform_dictionary["fl_x"] = fl_x
transform_dictionary["fl_y"] = fl_y
transform_dictionary["w"] = w
transform_dictionary["h"] = h
transform_dictionary["cx"] = cx
transform_dictionary["cy"] = cy
transform_dictionary["k1"] = k1
transform_dictionary["k2"] = k2
transform_dictionary["k3"] = k3
transform_dictionary["p1"] = p1
transform_dictionary["p2"] = p2
transform_dictionary["camera_model"] = camera_model
transform_dictionary["frames"] = frames

#Save as json 
with open(os.path.join(PROCESSED_DATA_PATH, "transforms.json"), 'w') as outfile:
    json.dump(transform_dictionary, outfile)
